### Singleton (Creational)
* Provides a mechanism to have one, and only one, object of a given type and provides a global point of access
* Used in cases such as logging or database operations, printer, and so on
* There is a need to have only one instance that is available across the application to avoid conflicting requests on the same resource

In [4]:
class Singleton(object):
    def __new__(cls):
        if not hasattr(cls,'instance'):
            cls.instance = super(Singleton,cls).__new__(cls)
        return cls.instance

############################

s0 = Singleton() 
print("Object created",s0)

s1 = Singleton()
print("Object created",s1)

Object created <__main__.Singleton object at 0x0680DF50>
Object created <__main__.Singleton object at 0x0680DF50>


* **\_\_new\_\_** Python’s special method to instantiate objects to control the object creation
* The **s0** object gets created with the **\_\_new\_\_** method, but before this, it checks whether the object already exist
* The **hasattr** method (Python’s special method to know if an object has a certain property) is used to see if the cls object has the instance property, which checks whether the class already has an object. 
* **super()** gives you access to methods in a superclass from the subclass that inherits from it. 
* Till the time the **s1** object is requested, **hasattr()** detects that an object already exists **s0** and hence **s1** allocates the existing object instance (located at the same address).

### Lazy Singleton
* in the case of module imports, we may accidently create an object even when it’s not needed. 
* makes sure that the object gets created when it’s actually needed.
* a way to work with reduced resources.

In [36]:
class LazySingleton:
    __instance = None
    def __init__(self):
        if not LazySingleton.__instance:
            print(" __init__ method called..")
        else:
            print("Instance already created:", self.getInstance())

    @classmethod
    def getInstance(cls):
        if not cls.__instance:
            cls.__instance = LazySingleton()
        return cls.__instance
    
############################

s0 = LazySingleton()  
print("Object created: ", LazySingleton.getInstance())  
s1 = LazySingleton()

 __init__ method called..
 __init__ method called..
Object created:  <__main__.LazySingleton object at 0x066B1CF0>
Instance already created: <__main__.LazySingleton object at 0x066B1CF0>


* **s=LazySingleton()**, it calls the **__init__** method but no new object gets created. 
* Actual object creation happens when we call **Singleton.getInstance()**
* **s1** instance already created

### Monostate Singleton pattern (Borg)
* there should be one and only one object of a class
* instances sharing the same state
* should be bothered about the state and behavior rather than the identity

In [25]:
class Borg:    
    __shared_state = {"rainy":"yes"}    
    def __init__(self):             
        self.__dict__ = self.__shared_state       
        pass
    
b = Borg()
b.traffic = 'yes'
print("Borg Object 'b': ", b) ## b and b1 are distinct objects
print("Object State 'b':", b.__dict__)
print('------')

b1 = Borg()
b1.weather = 'cold'

print("Borg Object 'b1': ", b1)
print("Object State 'b':", b.__dict__)## b and b1 share same state
print("Object State 'b1':", b1.__dict__)
print('------')

b2 = Borg()
b2.weather = 'fog'
b2.rainy = 'no'

print("Borg Object 'b2': ", b2) ## b, b1 and b2 are distinct objects
print("Object State 'b':", b.__dict__)## b, b1 and b2 share same state
print("Object State 'b1':", b1.__dict__)
print("Object State 'b2':", b2.__dict__)

Borg Object 'b':  <__main__.Borg object at 0x051BACF0>
Object State 'b': {'rainy': 'yes', 'traffic': 'yes'}
------
Borg Object 'b1':  <__main__.Borg object at 0x04415070>
Object State 'b': {'rainy': 'yes', 'traffic': 'yes', 'weather': 'cold'}
Object State 'b1': {'rainy': 'yes', 'traffic': 'yes', 'weather': 'cold'}
------
Borg Object 'b2':  <__main__.Borg object at 0x044159D0>
Object State 'b': {'rainy': 'no', 'traffic': 'yes', 'weather': 'fog'}
Object State 'b1': {'rainy': 'no', 'traffic': 'yes', 'weather': 'fog'}
Object State 'b2': {'rainy': 'no', 'traffic': 'yes', 'weather': 'fog'}


* **\_\_dict\_\_** variable to store the state of every object of a class
* **\_\_shared\_state** variable to store a default state (it could be any name)

### Metaclasses
* It is a class of a class
* The class is an instance of its metaclass. 
* To create classes of their own type from the predefined Python classes

In [40]:
class MyInt(type):    
    def __call__(cls, *args, **kwds):        
        print("***** Here's My int *****", args)        
        print("Now do whatever you want with these objects...")        
        return type.__call__(cls, *args, **kwds)
    
class int(metaclass=MyInt):    
    def __init__(self, x, y):        
        self.x = x        
        self.y = y

i = int(4,5)

***** Here's My int ***** (4, 5)
Now do whatever you want with these objects...


In [47]:
class MetaSingleton(type):    
    _instances = {}    
    def __call__(cls, *args, **kwargs):        
        if cls not in cls._instances:            
            cls._instances[cls] = super(MetaSingleton, cls).__call__(*args, **kwargs)        
            return cls._instances[cls]
        
class Logger(metaclass=MetaSingleton):    
    pass

logger1 = Logger()
logger2 = Logger()
logger3 = Logger()
print(logger1)
print(logger2)
print(logger3)

None
None


* when we instantiate the int class with **int(4,5)** 
* the **\_\_call\_\_** method of the **MyInt** metaclass gets called, 
* which means that the metaclass **MyInt** now controls the instantiation of the object
* metaclasses overridethe **\_\_new\_\_** and **\_\_init\_\_** method

### Example 1

In [49]:
import sqlite3

class MetaSingleton(type):    
    _instances = {}    
    def __call__(cls, *args, **kwargs):        
        if cls not in cls._instances:            
            cls._instances[cls] = super(MetaSingleton, cls).__call__(*args, **kwargs)        
        return cls._instances[cls]

class Database(metaclass=MetaSingleton):  
    connection = None  
    def connect(self):    
        if self.connection is None:        
            self.connection = sqlite3.connect("db.sqlite3")        
            self.cursorobj = self.connection.cursor()    
        return self.cursorobj
    
db1 = Database().connect()
db2 = Database().connect()
print ("Database Objects DB1", db1)
print ("Database Objects DB2", db2)

Database Objects DB1 <sqlite3.Cursor object at 0x0441CC20>
Database Objects DB2 <sqlite3.Cursor object at 0x0441CC20>


### Example 2

In [57]:
class HealthCheck:    
    _instance = None    
    def __new__(cls, *args, **kwargs):        
        if not HealthCheck._instance:            
            HealthCheck._instance = super(HealthCheck, cls).__new__(cls, *args, **kwargs)        
        return HealthCheck._instance    
    def __init__(self):        
        self._servers = []    
    def addServer(self,name):        
        self._servers.append("Server: " + name)            
    def changeLastServer(self,name):        
        self._servers.pop()        
        self._servers.append("Server: " + name)

hc1 = HealthCheck()
hc2 = HealthCheck()

hc1.addServer('Tokio')
hc1.addServer('Seoul')
hc1.addServer('Pekin')
hc1.addServer('Siem Reap')
print("Schedule health check for servers (1)..")
for i in range(len(hc1._servers)):    
    print("Checking ", hc1._servers[i])

hc2.changeLastServer('Kuala Lumpur')
print("Schedule health check for servers (2)..")
for i in range(len(hc1._servers)):    
    print("Checking ", hc2._servers[i])

Schedule health check for servers (1)..
Checking  Server: Tokio
Checking  Server: Seoul
Checking  Server: Pekin
Checking  Server: Siem Reap
Schedule health check for servers (2)..
Checking  Server: Tokio
Checking  Server: Seoul
Checking  Server: Pekin
Checking  Server: Kuala Lumpur
